In [ ]:
# Import libraries: cv2 (OpenCV), numpy, sys (1 point).
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def print_image(image, color_space="rgb", title=""):
    if len(image.shape) == 2:
        pixels = np.array(image)
        plt.imshow(pixels, cmap="gray")
    else:
        if color_space == "bgr":
            img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        elif color_space == "hsv":
            img = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
        else:
            img = image

        pixels = np.array(img)
        plt.imshow(pixels / 255.0)

    plt.title(title)
    plt.show()

In [ ]:
def print_image(image, color_space="rgb", title=""):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    # Sent image
    axes[0].imshow(image if len(image.shape) == 3 else image, cmap='gray' if len(image.shape) == 2 else None)
    axes[0].set_title("Raw Image")
    axes[0].axis("off")
    
    # Converted image
    if len(image.shape) == 2:
        converted_image = image
    else:
        if color_space == "bgr":
            converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        elif color_space == "hsv":
            converted_image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
        else:
            converted_image = image
    
    axes[1].imshow(converted_image / 255.0 if len(image.shape) == 3 else converted_image, cmap='gray' if len(image.shape) == 2 else None)
    axes[1].set_title("Converted Image")
    axes[1].axis("off")
    
    fig.suptitle(title)
    plt.show()

In [ ]:
# Import photo ball.png (1 point).
image = cv2.imread('red_ball.jpg')

# Set the condition for the correct loading of the image, e.g. using the 'sys.exit' command (1 point).
if image is None:
    sys.exit(1)

print_image(image, color_space='bgr')

In [ ]:
# Change the image format to HSV (1 point).
image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
print_image(image, color_space='hsv')

In [ ]:
# Find the colours using a binary operation (1 point).

# two masks as red is at the start (around 0) and the end (around 360) of the range 

# Hue 0 red, 20 orange; 360 red, 340 pinkish, divided by 2 because opencv has values 0-179
# Saturation quarter - max 65 - 255
# Value (brightness) quarter - max 65 - 255

sat = 50
val = 45

lower_red1 = np.array([0, sat, val])
upper_red1 = np.array([10, 255, 255])

lower_red2 = np.array([170, sat, val])
upper_red2 = np.array([180, 255, 255])

mask1 = cv2.inRange(image, lower_red1, upper_red1)
mask2 = cv2.inRange(image, lower_red2, upper_red2)

mask = mask1 + mask2

print_image(mask)

In [ ]:
# Improve image quality (remove noise) through morphological operations (1 point).

# https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html
# erosion "eats" the image, makes it smaller. 1 if ALL pixels under kernel are 1
# dilation "fattens" the image, makes it bigger. 1 if ANY pixel under kernel is 1

# opening - erosion followed by dilation, removes outer noise
# closing - dilation followed by erosion, removes inner noise

kernel = np.ones((7, 7), np.uint8)

# opening to remove noise in the background of the red ball (the white region)
mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
print_image(mask_cleaned)

In [ ]:
# closing to remove noise in the background of the red ball (the black region)
mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel)
print_image(mask_cleaned)

In [ ]:
image_labeled = image.copy()

In [ ]:
# image_labeled = image
# Add the calculated centre of gravity of the ball to the image (1 point).

# cv2.RETR_EXTERNAL retrieve only external contours
# cv2.CHAIN_APPROX_SIMPLE compress segments
contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if not contours:
    sys.exit(1)

largest_contour = max(contours, key=cv2.contourArea)

M = cv2.moments(largest_contour)
if M["m00"] != 0:
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    cv2.circle(image_labeled, (cx, cy), 3, (0, 0, 255), -1)
print_image(image_labeled, color_space='hsv')

In [ ]:
# Add the word "red ball" near the centre of gravity (1 point).
cv2.putText(image_labeled, "red ball", (cx - 40, cy - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
print_image(image_labeled, color_space='hsv')